## Projeto para criação do arquivo final em excel com votos válidos

### Imports ->

In [127]:
import pandas as pd
import plotly.express as px

### Funções ->

In [128]:
def criar_excel(df, caminho='dt_pandas_to_excel.xlsx', indice=True, nome_planilha='Planilha1'):
    '''Cria um arquivo do excel formatado a partir de um data frame pandas
    
    Parameters:
        df (objeto pandas DataFrame): Data frame a ser convertido em planilha do excel
        caminho (string): Caminho do local para salvar o arquivo, com nome do arquivo e extensão
        indice (boolean): Indica se o arquivo irá ou não criar coluna para os índices do data frame
        nome_planilha (string): Nome para a aba/planilha da pasta de trabalho do excel
    
    Returns:
        Não possui retorno    
    '''
    
    # Cria um escritor pandas excel usando XlsxWriter como mecanismo e indicando o caminho como parâmetro
    escritor = pd.ExcelWriter(caminho, engine='xlsxwriter')

    # Converte o data frame em um objeto excel XlsxWriter
    df.to_excel(
        escritor,
        sheet_name=nome_planilha, # Define o nome da aba/planilha da pasta de trabalho
        startrow=1, # Pula uma linha para que seja possível inserir cabeçalho personalizado
        header=False, # Desativa o cabeçalho padrão
        index=indice # Define se irá criar coluna para índice ou não de acordo com o parâmtero informado
    )

    # Obtém a pasta de trabalho XlsxWriter e os objetos de planilha
    pasta_trabalho = escritor.book # Pasta de trabalho
    pasta_trabalho = escritor.sheets[nome_planilha] # Planilha

    # Obtém as dimensões do data frame
    (qtde_linhas, qtde_colunas) = df.shape

    # Cria uma lista de cabeçalhos das colunas para uso no add_table()
    config_colunas = []
    if indice == True: # Se Parâmetro do índice for verdadeiro, inclui o primeiro cabeçalho como 'Índice'
        config_colunas.append({'header': 'Índice'})
    for cabecalho in df.columns: # Para cada coluna no data frame, inclui o cabeçalho na lista
        config_colunas.append({'header': cabecalho})
    
    # Caso parâmetro do índice for falso, diminui uma coluna da quantidade de colunas 
    if indice == False:
        qtde_colunas -= 1
        
    # Adiciona a tabela
    pasta_trabalho.add_table(0, 0, qtde_linhas, qtde_colunas, {'columns': config_colunas})

    # Aumente a largura das colunas para melhor visualização
    pasta_trabalho.set_column(0, qtde_colunas, 25)

    # Fecha o escritor pandas excel writer e salva o arquivo excel
    escritor.save()

### Abertura dos arquivos ->

In [129]:
# Abrir planilha de votos brutos
df_votos_brutos = pd.read_excel('votos_brutos_14-09-2021.xlsx')

# Abrir a planilha de códigos com os estabelecimentos preenchidos
df_codigos = pd.read_excel('codigos-categorias-14-09-2021.xlsx')

### Ajustes no data frame de códigos ->

In [130]:
# Alteração dos caracteres confusos do data frame de códigos
df_codigos['Código'] = df_codigos['Código'].str.replace('l', '1')
df_codigos['Código'] = df_codigos['Código'].str.replace('o', '0')

# Armazenando apenas as colunas de código, categoria e estabelecimento do data frame de códigos
df_codigos = df_codigos[['Código', 'Categoria', 'Estabelecimento']]

### Ajustes no data frame de votos ->

In [131]:
# Renomeando a coluna de códigos do data frame de votos, para igualar o nome com o do data frame de códigos
df_votos_brutos = df_votos_brutos.rename(columns={'Código de Validação':'Código'})

# Novo data frame de votos válidos recebe apenas os votos onde o código informado contiver 6 caracteres
df_votos_validos = df_votos_brutos[df_votos_brutos['Código'].str.len() == 6]

# Novo data frame de votos inválidos recebe apenas os votos onde o código informado não contiver 6 caracteres
df_votos_invalidos = df_votos_brutos[df_votos_brutos['Código'].str.len() != 6]

# Alterando todos os códigos de validação dos votos para lowercase (letras minúsculas)
# para caso alguém tenha digitado com alguma letra maiúscula por engano
df_votos_validos['Código'] = df_votos_validos['Código'].str.lower()

# Alteração dos caracteres confusos
df_votos_validos['Código'] = df_votos_validos['Código'].str.replace('l', '1')
df_votos_validos['Código'] = df_votos_validos['Código'].str.replace('o', '0')

# Excluir votos anteriores ao início do evento
df_votos_validos.drop(df_votos_validos.loc[df_votos_validos['Carimbo de data/hora'] < '2021-08-01 00:00:00'].index, inplace=True)

# Excluir votos com códigos inseridos mais de uma vez
df_votos_validos = df_votos_validos.drop_duplicates(subset=['Código'], keep='first')

<ipython-input-131-545d7217d33f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_votos_validos['Código'] = df_votos_validos['Código'].str.lower()
<ipython-input-131-545d7217d33f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_votos_validos['Código'] = df_votos_validos['Código'].str.replace('l', '1')
<ipython-input-131-545d7217d33f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

### Filtragem dos votos válidos por código ->

In [132]:
# Atribuir o valor da categoria e do estabelecimento para cada código do data frame de votos
df_votos_validos = df_votos_validos.merge(df_codigos, on='Código')

# Substituindo categoria de códigos trocados
df_votos_validos.loc[df_votos_validos['Estabelecimento'] == 'Bistrô Zen (usado como confeitaria)', 'Categoria'] = 'Confeitaria'

# Corrigindo nome do estabelecimento
df_votos_validos.loc[df_votos_validos['Estabelecimento'] == 'Bistrô Zen (usado como confeitaria)', 'Estabelecimento'] = 'Bistrô Zen'

# Ordenar as colunas recém inseridas após a coluna de códigos para melhor visualização
colunas_inicio = ['Carimbo de data/hora', 'Código', 'Categoria', 'Estabelecimento'] # Lista com as quatro primeiras colunas
todas_colunas = colunas_inicio + (df_votos_validos.columns.drop(colunas_inicio).tolist()) # Lista com todas as colunas ordenadas (as quatro primeiras e todas as colunas tirando as quatro primeiras)
df_votos_validos = df_votos_validos[todas_colunas] # Atribuindo o data frame à mesma variável com as colunas ordenadas

# Cria pasta de trabalho do excel com o data frame
criar_excel(df_votos_validos, caminho='df_votos_validos.xlsx', indice=False)
criar_excel(df_votos_invalidos, caminho='df_votos_invalidos.xlsx', indice=False)

### Repescagem votos inválidos ->

In [133]:
# Obtendo valores de código que possuem 7 caracteres
df_repescagem_strip = df_votos_invalidos[df_votos_invalidos['Código'].str.len() == 7]

# Excluindo espaços inseridos ao final dos códigos, resultando nos 6 caracteres
df_repescagem_strip['Código'] = df_repescagem_strip['Código'].str.strip()

# Alterando todos os códigos de validação dos votos para lowercase (letras minúsculas)
# para caso alguém tenha digitado com alguma letra maiúscula por engano
df_repescagem_strip['Código'] = df_repescagem_strip['Código'].str.lower()

# Alteração dos caracteres confusos
df_repescagem_strip['Código'] = df_repescagem_strip['Código'].str.replace('l', '1')
df_repescagem_strip['Código'] = df_repescagem_strip['Código'].str.replace('o', '0')

# Excluir votos com códigos inseridos mais de uma vez
df_repescagem_strip = df_repescagem_strip.drop_duplicates(subset=['Código'], keep='first')

# Atribuir o valor da categoria e do estabelecimento para cada código do data frame de votos
df_repescagem_strip = df_repescagem_strip.merge(df_codigos, on='Código')
criar_excel(df_repescagem_strip, caminho='df_repescagem_strip.xlsx', indice=False)

<ipython-input-133-f958b12edc16>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_repescagem_strip['Código'] = df_repescagem_strip['Código'].str.strip()
<ipython-input-133-f958b12edc16>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_repescagem_strip['Código'] = df_repescagem_strip['Código'].str.lower()
<ipython-input-133-f958b12edc16>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [134]:
# Obtendo valores de possíveis códigos na coluna de cupom fiscal
df_repescagem_cupom = df_votos_invalidos[df_votos_invalidos['Número do Cupom Fiscal'].str.len() == 6]

# Alterando o nome das colunas para fazer o merge
df_repescagem_cupom = df_repescagem_cupom.rename(columns={'Código':'Código Errado'})
df_repescagem_cupom = df_repescagem_cupom.rename(columns={'Número do Cupom Fiscal':'Código'})
df_repescagem_cupom = df_repescagem_cupom.rename(columns={'Código Errado':'Número do Cupom Fiscal'})

# Alterando todos os códigos de validação dos votos para lowercase (letras minúsculas)
# para caso alguém tenha digitado com alguma letra maiúscula por engano
df_repescagem_cupom['Código'] = df_repescagem_cupom['Código'].str.lower()

# Alteração dos caracteres confusos
df_repescagem_cupom['Código'] = df_repescagem_cupom['Código'].str.replace('l', '1')
df_repescagem_cupom['Código'] = df_repescagem_cupom['Código'].str.replace('o', '0')

# Excluir votos com códigos inseridos mais de uma vez
df_repescagem_cupom = df_repescagem_cupom.drop_duplicates(subset=['Código'], keep='first')

# Atribuir o valor da categoria e do estabelecimento para cada código do data frame de votos
df_repescagem_cupom = df_repescagem_cupom.merge(df_codigos, on='Código')
criar_excel(df_repescagem_cupom, caminho='df_repescagem_cupom.xlsx', indice=False)

In [135]:
# Juntando votos da repescagem
df_votos_repescagem = pd.concat([df_repescagem_strip, df_repescagem_cupom], ignore_index=True)
criar_excel(df_votos_repescagem, caminho='df_votos_repescagem.xlsx', indice=False)

# Juntando votos da repescagem com votos válidos
df_votos_validos_finais = pd.concat([df_votos_validos, df_votos_repescagem], ignore_index=True)

# Excluindo possíveis novas duplicatas
df_votos_validos_finais = df_votos_validos_finais.drop_duplicates(subset=['Código'], keep='first')
criar_excel(df_votos_validos_finais, caminho='df_votos_validos_finais.xlsx', indice=False)

## Gráficos ->

In [ ]:
# Aquisição de todas as categorias dos votos
categorias = list(df_votos_validos_finais['Categoria'].unique())

# Criação de dicionário para os data frames dos votos por categoria
votos_categoria = {}

# Atribuindo os votos filtrados por categoria ao dicionário
for categoria in categorias:
    votos_categoria[categoria] = df_votos_validos_finais[df_votos_validos_finais['Categoria'] == categoria]

In [176]:
# Criando série com o relatório de quantidades por categoria, com base no data frame de votos válidos final
relatorio_qtde = df_votos_validos_finais['Categoria'].value_counts()

# Transformando a série em data frame
relatorio_qtde = relatorio_qtde.to_frame(name='Quantidade')

# Resetando index da série
relatorio_qtde.reset_index(level=0, inplace=True)

# Alterando o nome da coluna
relatorio_qtde = relatorio_qtde.rename(columns = {'index' : 'Categoria'})

# Atribuindo o total de votos válidos a uma categoria
valor_total = int(relatorio_qtde['Quantidade'].sum())

# Criação do gráfico de pizza
fig = px.pie(relatorio_qtde, values='Quantidade', names='Categoria', title=f'Caraguá A Gosto Delivery - {valor_total} Votos Válidos')

# Ajustando os label das fatias
fig.update_traces(textinfo='percent+value')

# Exibindo o gráfico
fig.show()

In [177]:
relatorio_estab = {}

for categoria in votos_categoria:
    # Criando série com o relatório de quantidades da categoria por estabelecimento
    relatorio_estab[categoria] = votos_categoria[categoria]['Estabelecimento'].value_counts()
    
    # Transformando a série em data frame
    relatorio_estab[categoria] = relatorio_estab[categoria].to_frame(name='Quantidade')
    
    # Resetando index da série
    relatorio_estab[categoria].reset_index(level=0, inplace=True)

    # Alterando o nome da coluna
    relatorio_estab[categoria] = relatorio_estab[categoria].rename(columns = {'index' : 'Estabelecimento'})

    # Atribuindo o total de votos válidos a uma categoria
    valor_total = int(relatorio_estab[categoria]['Quantidade'].sum())

    # Criação do gráfico de pizza
    fig = px.pie(relatorio_estab[categoria], values='Quantidade', names='Estabelecimento', title=f'Caraguá A Gosto Delivery - {valor_total} Votos Válidos da Categoria {categoria}')

    # Ajustando os label das fatias
    fig.update_traces(textinfo='percent+value')

    # Exibindo o gráfico
    fig.show()